In [2]:
import requests
import pandas as pd
import json

In [54]:
# sql_query = f'SELECT "OBSERVATORY_NAME", AVG(CAST("AVERAGE_HUMIDITY" AS numeric)) AS AVERAGE_HUMIDITY FROM "TABLE_PLACEHOLDER" GROUP BY "OBSERVATORY_NAME"'
sql_query = '''
WITH day_table AS (
    SELECT EXTRACT(day FROM CAST("DATE_TIME" AS date)) AS "DAY", * 
    FROM "TABLE_PLACEHOLDER"
    LIMIT 100
)

SELECT "DAY", AVG(CAST("AVERAGE_HUMIDITY" AS numeric))::numeric(5,2) AS average_humidity
FROM day_table
GROUP BY "DAY"
ORDER BY "DAY"
'''

In [81]:
def run_sql(sql_query, table, fields=False):
    sql_query = sql_query.replace('TABLE_PLACEHOLDER',table)
    source = requests.get(f'https://data.ibb.gov.tr/api/3/action/datastore_search_sql?sql={sql_query}')
    try:
        result = pd.DataFrame(json.loads(source.text)['result']['records'])
        if fields:
            return pd.DataFrame(json.loads(source.text)['result']['fields'])
    except KeyError:
        result = json.loads(source.text)
    return result

In [219]:
columns = run_sql('SELECT * FROM "TABLE_PLACEHOLDER" LIMIT 10', "8ca0081a-333c-4e86-8ea2-ab8741bbfc95", True)['id'].tolist()

In [220]:
print(columns)

['_id', '_full_text', 'DATE_TIME', 'OBSERVATORY_NAME', 'SENSOR_TYPE', 'MINIMUM_TEMPERATURE', 'MAXIMUM_TEMPERATURE', 'AVERAGE_TEMPERATURE', 'MINIMUM_HUMIDITY', 'MAXIMUM_HUMIDITY', 'AVERAGE_HUMIDITY', 'MINIMUM_WIND', 'MAXIMUM_WIND', 'AVERAGE_WIND', 'MINIMUM_DIRECTIONOFWIND', 'MAXIMUM_DIRECTIONOFWIND', 'AVERAGE_DIRECTIONOFWIND', 'MINIMUM_PRECIPITATION', 'MAXIMUM_PRECIPITATION', 'AVERAGE_PRECIPITATION', 'MINIMUM_ROAD_TEMPERATURE', 'MAXIMUM_ROAD_TEMPERATURE', 'AVERAGE_ROAD_TEMPERATURE', 'MINIMUM_FELT_TEMPERATURE', 'MAXIMUM_FELT_TEMPERATURE', 'AVERAGE_FELT_TEMPERATURE']


In [221]:
col_string ='CAST ("' + '" AS numeric(5,2)), CAST ("'.join(columns[5:]) + '" AS numeric(5,2))'
print(col_string)

CAST ("MINIMUM_TEMPERATURE" AS numeric(5,2)), CAST ("MAXIMUM_TEMPERATURE" AS numeric(5,2)), CAST ("AVERAGE_TEMPERATURE" AS numeric(5,2)), CAST ("MINIMUM_HUMIDITY" AS numeric(5,2)), CAST ("MAXIMUM_HUMIDITY" AS numeric(5,2)), CAST ("AVERAGE_HUMIDITY" AS numeric(5,2)), CAST ("MINIMUM_WIND" AS numeric(5,2)), CAST ("MAXIMUM_WIND" AS numeric(5,2)), CAST ("AVERAGE_WIND" AS numeric(5,2)), CAST ("MINIMUM_DIRECTIONOFWIND" AS numeric(5,2)), CAST ("MAXIMUM_DIRECTIONOFWIND" AS numeric(5,2)), CAST ("AVERAGE_DIRECTIONOFWIND" AS numeric(5,2)), CAST ("MINIMUM_PRECIPITATION" AS numeric(5,2)), CAST ("MAXIMUM_PRECIPITATION" AS numeric(5,2)), CAST ("AVERAGE_PRECIPITATION" AS numeric(5,2)), CAST ("MINIMUM_ROAD_TEMPERATURE" AS numeric(5,2)), CAST ("MAXIMUM_ROAD_TEMPERATURE" AS numeric(5,2)), CAST ("AVERAGE_ROAD_TEMPERATURE" AS numeric(5,2)), CAST ("MINIMUM_FELT_TEMPERATURE" AS numeric(5,2)), CAST ("MAXIMUM_FELT_TEMPERATURE" AS numeric(5,2)), CAST ("AVERAGE_FELT_TEMPERATURE" AS numeric(5,2))


In [240]:
avg_colstring = ''
for col in columns[5:]:
    avg_colstring+= f'AVG("{col}")::numeric(5,2) AS "{col}",'
avg_colstring = avg_colstring[:-1]
avg_colstring

'AVG("MINIMUM_TEMPERATURE")::numeric(5,2) AS "MINIMUM_TEMPERATURE",AVG("MAXIMUM_TEMPERATURE")::numeric(5,2) AS "MAXIMUM_TEMPERATURE",AVG("AVERAGE_TEMPERATURE")::numeric(5,2) AS "AVERAGE_TEMPERATURE",AVG("MINIMUM_HUMIDITY")::numeric(5,2) AS "MINIMUM_HUMIDITY",AVG("MAXIMUM_HUMIDITY")::numeric(5,2) AS "MAXIMUM_HUMIDITY",AVG("AVERAGE_HUMIDITY")::numeric(5,2) AS "AVERAGE_HUMIDITY",AVG("MINIMUM_WIND")::numeric(5,2) AS "MINIMUM_WIND",AVG("MAXIMUM_WIND")::numeric(5,2) AS "MAXIMUM_WIND",AVG("AVERAGE_WIND")::numeric(5,2) AS "AVERAGE_WIND",AVG("MINIMUM_DIRECTIONOFWIND")::numeric(5,2) AS "MINIMUM_DIRECTIONOFWIND",AVG("MAXIMUM_DIRECTIONOFWIND")::numeric(5,2) AS "MAXIMUM_DIRECTIONOFWIND",AVG("AVERAGE_DIRECTIONOFWIND")::numeric(5,2) AS "AVERAGE_DIRECTIONOFWIND",AVG("MINIMUM_PRECIPITATION")::numeric(5,2) AS "MINIMUM_PRECIPITATION",AVG("MAXIMUM_PRECIPITATION")::numeric(5,2) AS "MAXIMUM_PRECIPITATION",AVG("AVERAGE_PRECIPITATION")::numeric(5,2) AS "AVERAGE_PRECIPITATION",AVG("MINIMUM_ROAD_TEMPERATURE")::

In [223]:
not_99 = ''
for col in columns[5:]:
    not_99 += f'"{col}" != -99 AND '
not_99 = not_99[:-5]
not_99

'"MINIMUM_TEMPERATURE" != -99 AND "MAXIMUM_TEMPERATURE" != -99 AND "AVERAGE_TEMPERATURE" != -99 AND "MINIMUM_HUMIDITY" != -99 AND "MAXIMUM_HUMIDITY" != -99 AND "AVERAGE_HUMIDITY" != -99 AND "MINIMUM_WIND" != -99 AND "MAXIMUM_WIND" != -99 AND "AVERAGE_WIND" != -99 AND "MINIMUM_DIRECTIONOFWIND" != -99 AND "MAXIMUM_DIRECTIONOFWIND" != -99 AND "AVERAGE_DIRECTIONOFWIND" != -99 AND "MINIMUM_PRECIPITATION" != -99 AND "MAXIMUM_PRECIPITATION" != -99 AND "AVERAGE_PRECIPITATION" != -99 AND "MINIMUM_ROAD_TEMPERATURE" != -99 AND "MAXIMUM_ROAD_TEMPERATURE" != -99 AND "AVERAGE_ROAD_TEMPERATURE" != -99 AND "MINIMUM_FELT_TEMPERATURE" != -99 AND "MAXIMUM_FELT_TEMPERATURE" != -99 AND "AVERAGE_FELT_TEMPERATURE" != -99'

In [234]:
sql_query = f'''
WITH temp_table AS (
SELECT DATE_TRUNC('day', "DATE_TIME"::date)::date AS "DATE", {col_string}
FROM "TABLE_PLACEHOLDER" 
LIMIT 100
),

no_99 AS (
SELECT CASE 
)

SELECT "DATE", {avg_colstring}
FROM temp_table
WHERE {not_99}
GROUP BY "DATE"
ORDER BY "DATE"
'''

In [239]:
run_sql(sql_query, "8ca0081a-333c-4e86-8ea2-ab8741bbfc95")

,AVERAGE_FELT_TEMPERATURE,AVERAGE_ROAD_TEMPERATURE,MINIMUM_FELT_TEMPERATURE,DATE,MINIMUM_PRECIPITATION,MAXIMUM_HUMIDITY,MAXIMUM_FELT_TEMPERATURE,MAXIMUM_PRECIPITATION,MAXIMUM_TEMPERATURE,AVERAGE_HUMIDITY,...,MINIMUM_TEMPERATURE,AVERAGE_TEMPERATURE,MAXIMUM_ROAD_TEMPERATURE,MINIMUM_HUMIDITY,MINIMUM_DIRECTIONOFWIND,MINIMUM_WIND,MAXIMUM_DIRECTIONOFWIND,MINIMUM_ROAD_TEMPERATURE,MAXIMUM_WIND,AVERAGE_PRECIPITATION
0,4.76,6.18,3.31,2020-01-01,0.00,74.83,6.99,0.00,6.73,70.44,...,6.19,6.40,7.10,68.32,1.00,0.93,352.00,5.50,9.09,0.00
1,4.70,2.46,2.92,2020-01-02,0.00,95.20,5.78,0.00,3.46,93.83,...,2.77,3.00,2.70,92.40,17.00,0.00,349.00,2.30,0.73,0.00
2,5.55,5.25,4.36,2020-01-04,0.00,74.50,6.39,0.00,5.69,71.55,...,5.38,5.55,5.43,69.13,56.00,1.05,311.00,5.05,2.39,0.00
3,2.78,5.71,1.43,2020-01-05,0.68,96.60,4.21,1.24,5.74,96.05,...,5.50,5.62,5.80,95.60,1.00,2.21,355.00,5.70,9.41,0.92
4,-0.27,4.11,-1.05,2020-01-07,0.00,92.10,0.88,0.01,5.04,91.15,...,4.93,4.98,4.20,90.50,8.00,8.94,47.00,4.10,18.63,0.00
5,5.53,4.99,3.85,2020-01-08,0.00,80.40,6.78,0.00,4.49,79.82,...,4.24,4.33,5.05,78.51,5.00,0.11,352.50,4.85,1.70,0.00
6,1.19,2.51,0.31,2020-01-10,0.00,95.60,2.59,0.00,2.38,93.44,...,1.32,1.96,2.80,91.70,204.00,0.92,258.00,2.20,1.98,0.00
7,6.96,9.06,6.07,2020-01-12,0.00,99.70,8.20,0.00,8.40,99.56,...,7.99,8.22,9.50,99.50,6.00,1.97,352.00,8.50,5.17,0.00
8,6.39,8.94,5.11,2020-01-13,0.00,83.77,7.87,15.00,7.48,81.28,...,6.47,7.05,10.23,79.11,17.00,0.93,102.25,7.98,3.64,7.87
9,4.08,3.01,3.02,2020-01-14,0.00,100.00,4.95,0.00,4.92,100.00,...,4.44,4.69,3.40,100.00,38.00,1.01,107.00,2.70,3.17,0.00


In [262]:
col_string = ''
for col in columns[5:]:
    col_string+= f'(CASE "{col}" WHEN \'-99\' THEN \'0\' ELSE "{col}" END) AS "{col}",'
col_string

'(CASE "MINIMUM_TEMPERATURE" WHEN \'-99\' THEN \'0\' ELSE "MINIMUM_TEMPERATURE" END) AS "MINIMUM_TEMPERATURE",(CASE "MAXIMUM_TEMPERATURE" WHEN \'-99\' THEN \'0\' ELSE "MAXIMUM_TEMPERATURE" END) AS "MAXIMUM_TEMPERATURE",(CASE "AVERAGE_TEMPERATURE" WHEN \'-99\' THEN \'0\' ELSE "AVERAGE_TEMPERATURE" END) AS "AVERAGE_TEMPERATURE",(CASE "MINIMUM_HUMIDITY" WHEN \'-99\' THEN \'0\' ELSE "MINIMUM_HUMIDITY" END) AS "MINIMUM_HUMIDITY",(CASE "MAXIMUM_HUMIDITY" WHEN \'-99\' THEN \'0\' ELSE "MAXIMUM_HUMIDITY" END) AS "MAXIMUM_HUMIDITY",(CASE "AVERAGE_HUMIDITY" WHEN \'-99\' THEN \'0\' ELSE "AVERAGE_HUMIDITY" END) AS "AVERAGE_HUMIDITY",(CASE "MINIMUM_WIND" WHEN \'-99\' THEN \'0\' ELSE "MINIMUM_WIND" END) AS "MINIMUM_WIND",(CASE "MAXIMUM_WIND" WHEN \'-99\' THEN \'0\' ELSE "MAXIMUM_WIND" END) AS "MAXIMUM_WIND",(CASE "AVERAGE_WIND" WHEN \'-99\' THEN \'0\' ELSE "AVERAGE_WIND" END) AS "AVERAGE_WIND",(CASE "MINIMUM_DIRECTIONOFWIND" WHEN \'-99\' THEN \'0\' ELSE "MINIMUM_DIRECTIONOFWIND" END) AS "MINIMUM_DIRE

In [263]:
sql_query = f"""
SELECT {col_string[:-1]} FROM "TABLE_PLACEHOLDER" LIMIT 10
"""

In [264]:
run_sql(sql_query, "8ca0081a-333c-4e86-8ea2-ab8741bbfc95")

,AVERAGE_ROAD_TEMPERATURE,AVERAGE_DIRECTIONOFWIND,MAXIMUM_PRECIPITATION,MINIMUM_DIRECTIONOFWIND,MINIMUM_ROAD_TEMPERATURE,AVERAGE_FELT_TEMPERATURE,AVERAGE_PRECIPITATION,MINIMUM_WIND,MAXIMUM_FELT_TEMPERATURE,MINIMUM_FELT_TEMPERATURE,...,MAXIMUM_DIRECTIONOFWIND,AVERAGE_TEMPERATURE,MAXIMUM_TEMPERATURE,MAXIMUM_ROAD_TEMPERATURE,MINIMUM_PRECIPITATION,MAXIMUM_HUMIDITY,MAXIMUM_WIND,MINIMUM_HUMIDITY,AVERAGE_WIND,AVERAGE_HUMIDITY
0,0,138.66667,0,127,0,5.2765,0,2.8,5.61,4.89,...,153,6.87,7,0,0,100,3.8,100,3.27667,100
1,13.47759,72.03448,0,41,12,5.73224,0,3.17,6.74,4.99,...,113,7.92962,8.34,14.7,0,79.85,6.13,70.83,5.05552,75.62017
2,0,179.0339,0,0,0,2.09627,0,2.1,3.53,0.91,...,359,4.78305,5.1,0,0,86,7.5,80,4.60847,84.37288
3,0,76.45,0,7,0,9.1565,0,0.3,11.52,7.69,...,359,9.07333,9.2,0,0,76,3.6,72,1.68167,75.01667
4,0,125.71186,0.2,0,0,1.33932,0.00339,2.1,3.04,-0.51,...,359,4.18475,4.3,0,0,91,9.7,87,4.68983,89.0678
5,0,0,0,0,0,0,0,0,0,0,...,0,4.75833,5.5,0,0,90,0,90,0,90
6,7.485,185.98333,0,3,6.9,8.52583,0,0.496,9.43,7.05,...,325,7.81592,7.985,7.9,0,63.51,1.864,59.26,0.9819,61.57033
7,0,139.50877,0,2,0,2.26456,0,2.4,4.04,0.64,...,359,5.27719,5.5,0,0,89,9.2,80,5.68596,84.42105
8,7.4,244.98333,0,146,7.4,7.47333,0,1.17,8.97,5.31,...,312,7.83047,8.08,7.4,0,75.41,3.65,72.96,2.19533,74.33883
9,6.17667,97.18333,0,1,5.5,4.75783,0,0.93,6.99,3.31,...,352,6.40102,6.73,7.1,0,74.83,9.09,68.32,3.526,70.44233


In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time

In [ ]:
source = requests.get()